<a href="https://colab.research.google.com/github/mamsdiallo/GenerativeAI/blob/main/Chatbot_project_MDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

I Choose the new dataset: "2022_in_science" from Wikipedia
It could be found here: https://en.wikipedia.org/wiki/2022_in_science

Rationale: It contains scientific events occurred in 2022. It is appropropriate since it is comprehensive and contains dated facts. Customization could be demonstrated with such dataset.

Important notice:
From January to March, the page contains only a link to the dataset stored in a dedicated page "January–March 2022 in science". It could be found here: https://en.wikipedia.org/wiki/January%E2%80%93March_2022_in_science#January

To get the full dataset, we should also retrieve also this webpage.

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [1]:
!pip install openai==0.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.3 MB/s eta 0:00:00


In [2]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.8 MB/s eta 0:00:00


In [3]:
import openai
openai.api_key = "YOUR API KEY"

In [4]:
from dateutil.parser import parse
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Get the Wikipedia page for "2022" since OpenAI's models stop in 2021
#resp = requests.get("https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exlimit=1&titles=2022&explaintext=1&formatversion=2&format=json")
resp = requests.get("https://en.wikipedia.org/wiki/2022_in_science") # retrieve as json

soup = BeautifulSoup(resp.content, 'html.parser')

In [5]:
# Write the output to a file: To bypass the output limits entirely of print(soup.prettify())
with open("output.html", "w") as file:
    file.write(soup.prettify())

In [ ]:
# Use HTML visualization tools
from IPython.core.display import display, HTML

display(HTML(soup.prettify()))


In [7]:
import pandas as pd

In [8]:
# Get the Wikipedia page for "January–March_2022_in_science" since OpenAI's models stop in 2021
# it is the first 3 months in 2022 as explained at the beginning of the notebook
# we'll then concatenate with the complete dataset "2022 in science"
params = {
    "action": "query",
    "prop": "extracts",
    "exlimit": 1,
    "titles": "January–March_2022_in_science",
    "explaintext": 1,
    "formatversion": 2,
    "format": "json"
}

In [9]:
resp = requests.get("https://en.wikipedia.org/w/api.php", params=params)

# Load page text into a dataframe
dfJan2March = pd.DataFrame()
dfJan2March["text"] = resp.json()["query"]["pages"][0]["extract"].split("\n")

# Clean up text to remove empty lines and headings
dfJan2March = dfJan2March[(dfJan2March["text"].str.len() > 0)]

# Logic to reformat the death entries as specified
death_section = False
for i, row in dfJan2March.iterrows():
    if row['text'] == "== Deaths ==":
        death_section = True
    elif row['text'] == "== See also ==":
        death_section = False
    elif death_section:
        date_part, name_part = row['text'].split(" – ", 1)
        dfJan2March.at[i, 'text'] = f"{date_part} – Death of {name_part}"

In [10]:
# Clean up text to remove empty lines and headings
dfJan2March = dfJan2March[(dfJan2March["text"].str.len() > 0) & (~dfJan2March["text"].str.startswith("=="))]

In [11]:
# In some cases dates are used as headings instead of being part of the
# text sample; adjust so dated text samples start with dates
prefix = ""
for (i, row) in dfJan2March.iterrows():
    # If the row already has " - ", it already has the needed date prefix
    if " – " not in row["text"]:
        try:
            # If the row's text is a date, set it as the new prefix
            parse(row["text"])
            prefix = row["text"]
        except:
            # If the row's text isn't a date, add the prefix
            row["text"] = prefix + " – " + row["text"]
dfJan2March = dfJan2March[dfJan2March["text"].str.contains(" – ")]
dfJan2March

,text
0,– This article lists a number of significant ...
7,"1 January – Israel reports, for the first time..."
9,"3 January – OpenAlex, a free online index of o..."
10,5 January – Scientists show how antibiotic res...
12,6 January – The global atmospheric methane con...
...,...
174,31 March – 2020s#Global goals and issues
175,31 March – Category:Science events
176,31 March – Category:Science timelines
177,31 March – List of emerging technologies


In [12]:
# Drop the first row since no relevant information is provided
dfJan2March = dfJan2March.drop(dfJan2March.index[0])

In [13]:
# Get the Wikipedia page for "2022 in science" since OpenAI's models stop in 2021
params = {
    "action": "query",
    "prop": "extracts",
    "exlimit": 1,
    "titles": "2022 in science",
    "explaintext": 1,
    "formatversion": 2,
    "format": "json"
}

In [14]:
resp = requests.get("https://en.wikipedia.org/w/api.php", params=params)

# Load page text into a dataframe
df = pd.DataFrame()
df["text"] = resp.json()["query"]["pages"][0]["extract"].split("\n")

# Clean up text to remove empty lines and headings
df = df[(df["text"].str.len() > 0)]

# Logic to reformat the death entries as specified
death_section = False
for i, row in df.iterrows():
    if row['text'] == "== Deaths ==":
        death_section = True
    elif row['text'] == "== See also ==":
        death_section = False
    elif death_section:
        date_part, name_part = row['text'].split(" – ", 1)
        df.at[i, 'text'] = f"{date_part} – Death of {name_part}"

In [15]:
# Clean up text to remove empty lines and headings
df = df[(df["text"].str.len() > 0) & (~df["text"].str.startswith("=="))]

In [16]:
# In some cases dates are used as headings instead of being part of the
# text sample; adjust so dated text samples start with dates
prefix = ""
for (i, row) in df.iterrows():
    # If the row already has " - ", it already has the needed date prefix
    if " – " not in row["text"]:
        try:
            # If the row's text is a date, set it as the new prefix
            parse(row["text"])
            prefix = row["text"]
        except:
            # If the row's text isn't a date, add the prefix
            row["text"] = prefix + " – " + row["text"]
df = df[df["text"].str.contains(" – ")]
df

,text
0,– The following scientific events occurred in...
17,1 April – Biochemists report finishing the com...
18,"1 April – A study shows that, contrary to wide..."
20,4 April – The Intergovernmental Panel on Clima...
21,4 April – Researchers announce a new technique...
...,...
581,22 December – Category:Science events
582,22 December – Category:Science timelines
583,22 December – List of emerging technologies
584,22 December – List of years in science


In [17]:
# Drop the first row since no relevant information is provided
df = df.drop(df.index[0])

In [18]:
combined_df = pd.concat([dfJan2March, df], ignore_index=True)

In [19]:
combined_df

,text
0,"1 January – Israel reports, for the first time..."
1,"3 January – OpenAlex, a free online index of o..."
2,5 January – Scientists show how antibiotic res...
3,6 January – The global atmospheric methane con...
4,6 January – Astronomers report the first direc...
...,...
522,22 December – Category:Science events
523,22 December – Category:Science timelines
524,22 December – List of emerging technologies
525,22 December – List of years in science


In [20]:
# Find duplicates
duplicates = combined_df.duplicated(keep='first')


In [21]:
# Display duplicates
df_duplicates = combined_df[duplicates]
print(df_duplicates)


                                                  text
494  18 January – Death of Sir David Cox, English s...
495  15 March – Death of Eugene Parker, American so...
496  20 March – Death of Wen Shengchang, Chinese oc...
497  23 March – Death of Arthur Riggs, American gen...
498  27 March – Death of Martin Pope, American phys...
499  27 March – Death of James Vaupel, American dem...
500  29 March – Death of Paul Benioff, American phy...
501  30 March – Death of Kenneth Walters, British m...


In [22]:
unique_df = combined_df.drop_duplicates(keep='first')
unique_df

,text
0,"1 January – Israel reports, for the first time..."
1,"3 January – OpenAlex, a free online index of o..."
2,5 January – Scientists show how antibiotic res...
3,6 January – The global atmospheric methane con...
4,6 January – Astronomers report the first direc...
...,...
522,22 December – Category:Science events
523,22 December – Category:Science timelines
524,22 December – List of emerging technologies
525,22 December – List of years in science


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [23]:
# Generating Embeddings
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(unique_df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=unique_df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )

    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
#unique_df["embeddings"] = embeddings
unique_df.loc[:, "embeddings"] = embeddings
unique_df

<ipython-input-23-37fe248bf3c3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df.loc[:, "embeddings"] = embeddings


,text,embeddings
0,"1 January – Israel reports, for the first time...","[-0.007021766621619463, -0.007521929685026407,..."
1,"3 January – OpenAlex, a free online index of o...","[0.0030271480791270733, -0.0010666691232472658..."
2,5 January – Scientists show how antibiotic res...,"[0.010954593308269978, -0.005460788495838642, ..."
3,6 January – The global atmospheric methane con...,"[0.005663083866238594, 0.009484306909143925, -..."
4,6 January – Astronomers report the first direc...,"[0.000807469361461699, -0.007914595305919647, ..."
...,...,...
522,22 December – Category:Science events,"[-0.005128823220729828, 0.004400391131639481, ..."
523,22 December – Category:Science timelines,"[-0.012002752162516117, -0.0026352410204708576..."
524,22 December – List of emerging technologies,"[-0.008796149864792824, 0.0012971083633601665,..."
525,22 December – List of years in science,"[-0.0005206000641919672, 0.007003664039075375,..."


In [24]:
# see case study
from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """

    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)

    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )

    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

In [25]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")

    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context:

{}

---

Question: {}
Answer:"""

    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))

    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:

        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count

        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

In [26]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model

    If the model produces an error, return an empty string
    """

    prompt = create_prompt(question, df, max_prompt_tokens)

    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

In [27]:
def interactive_question_session():
    while True:  # This loop will run indefinitely until 'break' is encountered.
        user_input = input("Type your question or 'exit' to quit: ")
        if user_input.lower() == 'exit':
            print("Exiting the session.")
            break  # Exits the loop
        else:
            # Here you can add code to process the user's question.
            # For example, calling a function that handles the question.

            prompt = "Question: "+user_input+ "\nAnswer:"
            print("You asked:", prompt)

            # text-davinci-003 is deprecated
            # and is replaced with gpt-3.5-turbo-instruct
            basic_answer = openai.Completion.create(
                model="gpt-3.5-turbo-instruct",
                prompt=prompt
            )

            print("basic answer:",basic_answer["choices"][0]["text"])

            # The custo answer
            custo_answer = answer_question(user_input, unique_df)
            print("Custo Answer",custo_answer)

# To start the interactive session:
interactive_question_session()


Type your question or 'exit' to quit: Who are the winners of the last Nobel prize to your knowledge in Physics and when?
You asked: Question: Who are the winners of the last Nobel prize to your knowledge in Physics and when?
Answer:
basic answer:  The winners of the last Nobel Prize in Physics, awarded in 2020,
Custo Answer Alain Aspect, John F. Clauser and Anton Zeilinger were awarded the Nobel Prize in Physics in October 5, 2022.
Type your question or 'exit' to quit: Who are the winner of the Fields Medal in mathematics to your knowledge and when?
You asked: Question: Who are the winner of the Fields Medal in mathematics to your knowledge and when?
Answer:
basic answer:  Some well-known winners of the Fields Medal include Terence Tao (2006),
Custo Answer Hugo Duminil-Copin, June Huh, James Maynard, and Maryna Viazovska won the Fields Medal in mathematics in 5 July.
Type your question or 'exit' to quit: Where did the last Climate Change Conference took place?
You asked: Question: Wher

### Question 1

In [28]:
nobel_prompt = """
Question: Who are the winners of the last Nobel prize to your knowledge in Physics and when?
Answer:
"""
# answer from the basic completion model - oldest for this model is in 2021
# Frequent answers are:
# in 2020
# Roger Penrose "for the discovery that black hole formation is a robust prediction of the general theory of relativity"
# Reinhard Genzel & Andrea M. Ghez "for the discovery of a supermassive compact object at the centre of our galaxy"
# in 2021
# Syukuro Manabe & Klaus Hasselmann "for the physical modelling of Earth's climate, quantifying variability and reliably predicting global warming"
# Giorgio Parisi "for the discovery of the interplay of disorder and fluctuations in physical systems from atomic to planetary scales"

In [29]:
# The custo answer should provide the latest information - after 2020
# Alain Aspect, John F. Clauser, and Anton Zeilinger were awarded the Nobel Prize in Physics on October 4, 2022.

### Question 2

In [30]:
fields_prompt = """
Question: Who are the winner of the Fields Medal in mathematics to your knowledge and when?
Answer:
"""
# frequent answer: As an AI, I don't have personal knowledge or the ability to keep track

In [31]:
# The custo answer should provide the latest information for the Fields Medal occuring in 2022
# Correct answer: The winners of the last Fields Medal in mathematics are Hugo Duminil-Copin, June Huh, James Maynard, and Maryna Viazovska. They were awarded the medal on 5 July.

### Question 3

In [32]:
climate_prompt = """
Question: Where did the last Climate Change Conference took place?
Answer:
"""
# Answer from the basic model: it could provide partial answers
# frequent answers:
# The 26th UN Climate Change Conference (COP26) took place from 31 October to 12 November 2021, in Glasgow, Scotland
# The 2019 United Nations Climate Change Conference (COP 25) took


In [33]:
# The custo answer should provide more up to date information for the Climate Conference occuring in 2022
# Correct answer: The 2022 United Nations Climate Change Conference (COP27) took place in Sharm el-Sheikh, Egypt.

### Conclusion
The model responds the differently meaning the dataset was appropriate for the task